In [35]:
using LinearAlgebra
using SparseArrays
include("arnoldi.jl")

arnoldi (generic function with 1 method)

In [36]:
n = 100
M = 40
L = 2
# We need this condition for now
M * L <= n

true

In [37]:
#^^^^^^^^^^^^^^^^^^^^^^^#
#  A*X_σ - X_σ * D = B  #
#_______________________#
# LHS
#   A
λ = [i+10 for i in 1:n]
a = randn(n, n)
A = UpperTriangular(a) - diagm(diag(a)) + diagm(λ)
#   Shift
σ = rand(1:9, L)
D = diagm(σ) 
Γ = .7
# RHS
B = randn(n, L);

In [38]:
x_exact = zeros(n, L)
for l=1:L
    lhs = A - D[l, l]*I
    b = B[:, l]
    x_exact[:, l] = lhs \ b
end

In [39]:
# Naive: Standard Arnoldi in a for-loop
#   [A](nxn) x = b
#   [Q](n x m+1) = [q1 q2 ... q(m+1)], q1 = normalized b
#   [H](m+1 x m): upper Hessenberg
#   A Q_m = Q H , [Q_m](nxm) = [q1 q2 ... qm]
#   x = Q_m z
#   A Q_m z = b = Q H z
#   Q' b = Q' Q H z => H z = Q' b = s => z = H \ s => x = Q_m z
x_naive = zeros(n, L) 
for l=1:L
    lhs = A - D[l, l]*I
    b = B[:, l]
    Q, H = arnoldi(lhs, b, M)
    for m in 1:M # Changed the following to be consistent with bArnoldi and gives smaller norms
        s = vcat(norm(b), zeros(m-1, 1)) #   size m+1 -> m
        z = H[1:m, 1:m] \ s              #   H[1:m+1, 1:m] -> H[1:m, 1:m]
        x = Q[:, 1:m] * z
        x_naive[:, l] = x
    end
end

In [40]:
norm(x_naive - x_exact)

1.0088272673390422e-8